<a href="https://colab.research.google.com/github/gala127/My_Important_Dates/blob/master/Voice_Telegram_Bot_v_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import psycopg2
import requests
from datetime import datetime
# import telebot
# from gtts import gTTS
# from io import BytesIO
# import whisper
import random
import time
import numpy as np
import soundfile as sf

# Telegram bot initialization (disabled for testing)
# TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN", "your-telegram-token")
# bot = telebot.TeleBot(TELEGRAM_TOKEN)

# Whisper model (disabled for testing)
# model = whisper.load_model("base")

# ChatGPT 4.1 API access
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-key")
CHATGPT_API_URL = "https://api.openai.com/v1/chat/completions"

def chatgpt_respond(messages):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API_KEY}"
    }
    data = {
        "model": "gpt-4-1106-preview",
        "messages": messages,
        "temperature": 0.7
    }
    response = requests.post(CHATGPT_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"[Ошибка от ChatGPT: {response.status_code}]"

# PostgreSQL database initialization (ensure table exists)
def ensure_db():
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS consultations (
        id SERIAL PRIMARY KEY,
        nickname TEXT,
        password TEXT,
        question TEXT[],
        answer TEXT[],
        emotion TEXT,
        intensity TEXT,
        started_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        summary TEXT
    )
    """)
    conn.commit()
    cur.close()
    conn.close()

# Connect to PostgreSQL
def connect_db():
    return psycopg2.connect(
        dbname=os.getenv('POSTGRES_DB', 'consulting'),
        user=os.getenv('POSTGRES_USER', 'postgres'),
        password=os.getenv('POSTGRES_PASSWORD', 'postgres'),
        host=os.getenv('POSTGRES_HOST', 'localhost'),
        port=os.getenv('POSTGRES_PORT', '5432')
    )

# Save full Q&A session to database

def save_step1_to_db(state):
    conn = connect_db()
    cur = conn.cursor()
    cur.execute("""
    INSERT INTO consultations (nickname, password, question, answer, emotion, intensity, started_at, summary)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        state.get('nickname'),
        state.get('password'),
        state.get('qa_questions'),
        state.get('qa_answers'),
        state.get('last_emotion'),
        state.get('last_intensity'),
        state.get('started_at'), state.get('summary')
    ))
    conn.commit()
    cur.close()
    conn.close()

# Emotion analysis and voice/text response (text-only for testing)

def analyze_emotion_intensity(audio_path):
    return "тестовая интенсивность"

def detect_text_emotion(text):
    keywords = {
        "уныние": ["не хочется", "устал", "бессмысленно"],
        "тревога": ["боюсь", "переживаю", "беспокойство"],
        "злость": ["раздражает", "злюсь", "невыносимо"],
        "радость": ["нравится", "хорошо", "доволен"],
        "спокойствие": ["спокойно", "устроено", "нормально"]
    }
    for emotion, words in keywords.items():
        for word in words:
            if word in text:
                return emotion
    return "неопределено"

# Stub for sending response (print instead of sending to Telegram)
def send_text_and_voice(chat_id, text):
    text = text.replace("без эмоций", "если возможно")
    text = text.replace("Спасибо", "")
    print(text.strip())

# Generate summary from collected answers using GPT

def generate_summary_from_step1(state):
    input_summary = (
        f"Клиентка сообщила, что ее
         муж не хочет зарабатывать деньги. "
        f"Это чаще всего проявляется {state.get('qa_answers')[1].lower()}, "
        f"особенно {state.get('qa_answers')[2].lower()}. "
        f"По мнению клиентки
        , причина в том, что {state.get('qa_answers')[3].lower()}."
    )

    messages = [
        {
            "role": "system",
            "content": (
                "Ты опытный психолог-консультант по методике Суркова. На основе описания клиента "
                "сформулируй суть проблемы в одной фразе. Формулировка должна содержать: кто, где, когда, "
                "в чём выражается затруднение и предполагаемую причину, если она названа. Не используй списков, "
                "избегай эмоциональных оценок. Тон спокойный, деловой."
            )
        },
        {"role": "user", "content": input_summary}
    ]

    return chatgpt_respond(messages)

# Simulated session start
if __name__ == '__main__':
    ensure_db()
    print("Бот запущен в тестовом режиме (без Telegram и голоса)")

    # Пример тестового состояния
    state = {
        'nickname': 'demo_user',
        'password': 'demo_pass',
        'qa_questions': [
            "Что именно происходит не так, как хотелось бы?",
            "Где именно проявляется проблема?",
            "Когда особенно ощущается проблема?",
            "Почему, по вашему мнению, это происходит?"
        ],
        'qa_answers': [
            "Мой муж не хочет зарабатывать деньги",
            "Сильнее всего в магазине или в онлайн-магазине",
            "В любой ситуации, когда я делаю покупки, днем, вечером, в выходные",
            "Я думаю, лень и усталость"
        ],
        'last_emotion': 'уныние',
        'last_intensity': 'средняя',
        'started_at': datetime.now()
    }

    summary = generate_summary_from_step1(state)
    state['summary'] = summary
    save_step1_to_db(state)
